In [1]:
import pandas as pd
import json
import string
# from textblob import TextBlob
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer 


import nltk
from nltk import sent_tokenize
from nltk import word_tokenize
import unicodedata
import re
from cleanco import basename
from thefuzz import fuzz
from name_matching.name_matcher import NameMatcher

### Lemmatize the sources (record labels)

In [3]:
# nltk.download('wordnet')

In [4]:
# !pip install cleanco
# !pip install thefuzz
# !pip install name_matching

In [5]:
all_files_za = pd.read_csv("Classification/all_files_globalv1.csv")

In [6]:
all_sources = all_files_za["source"].tolist()

In [7]:
# set(all_sources)

In [8]:
tokens_words = nltk.word_tokenize(all_sources[0])
print(tokens_words)

['Ye']


In [9]:
ps = PorterStemmer()
lemmatizer = WordNetLemmatizer()

In [10]:
lemmed_sources = []
for aword in all_sources:
    tokens_words = nltk.word_tokenize(aword)
    the_sentence = ""
    for singleword in tokens_words:
        new_word = ps.stem(singleword)
        new_word = lemmatizer.lemmatize(new_word)
        new_word = unicodedata.normalize('NFKD', new_word).encode('ASCII', 'ignore').decode()
        new_word = re.sub(r'[^\w\s]','',new_word)
        new_word = basename(new_word)
        the_sentence = the_sentence + " " + new_word
        the_sentence = the_sentence.lstrip()
    lemmed_sources.append(the_sentence)

In [11]:
all_files_za["lemmed sources"] = lemmed_sources

In [97]:
# all_files_za[all_files_za["lemmed sources"] == "300 entertain"][["source", "lemmed sources"]]

In [14]:
unique_lemmed = all_files_za["lemmed sources"].unique().tolist()
unique_source = all_files_za["source"].unique().tolist()

In [15]:
len(unique_lemmed), len(unique_source)

(1138, 1149)

In [16]:
# all_files_za.to_csv("all_files_za_lem.csv", index=False)

### Do fuzzy matching

In [18]:
matcher = NameMatcher(top_n=10,
    lowercase=True,
    punctuations=True,
    remove_ascii=True,
    legal_suffixes=False,
    common_words=False,
    verbose=True)

matcher.set_distance_metrics(['discounted_levenshtein','SSK', 'fuzzy_wuzzy_token_sort'])

In [19]:
all_files_za1 = all_files_za[["source","lemmed sources"]]
all_files_za1 = all_files_za1.drop_duplicates()

In [20]:
# matcher.load_and_process_master_data('lemmed sources', all_files_za1)

In [21]:
# dfmatch = matcher.match_names(to_be_matched=all_files_za1, column_matching='lemmed sources')

In [22]:
# dfmatch.to_csv("dfmatch.csv", index=False)

In [23]:
all_files_za1["sourcenew"]=all_files_za1["source"].apply(lambda x : x.replace(",", ""))

In [24]:
all_files_za1["source_split"]=all_files_za1["sourcenew"].apply(lambda x : x.split("/"))

In [25]:
all_files_za1["source_len"] = all_files_za1["source_split"].apply(lambda x : len(x))

In [26]:
all_files_za1["source_len"].unique().tolist()

[1, 2, 4, 7, 3, 6]

In [27]:
all_files_za1[['source0', 'source1', 'source2', 'source3', 'source4', 'source5', 'source6']] = pd.DataFrame(all_files_za1['source_split'].tolist(), index=all_files_za1.index)

In [28]:
all_files_za1.drop(["lemmed sources","source_split", "sourcenew"],inplace=True,axis=1)

In [29]:
all_files_zavf = all_files_za.merge(all_files_za1, left_on=['source'], right_on=['source'],how="left")

In [30]:
all_files_zavf.to_csv("all_files_globalvf.csv",index=False)

In [93]:
all_files_zavf

,trackAppearanceCount,artistAppearanceCount,artist_names,track_name,rank,uri,source,peak_rank,previous_rank,weeks_on_chart,...,IsBoyGroup,lemmed sources,source_len,source0,source1,source2,source3,source4,source5,source6
0,0,0,"$, Kanye West, Ty Dolla $ign",BACK TO ME,24,spotify:track:1icgLGTpX2fQXKRe4D7w2b,Ye,24,-1,1,...,False,ye,1,Ye,None,None,None,None,None,None
1,1,16,"$, Kanye West, Ty Dolla $ign",BACK TO ME,81,spotify:track:1icgLGTpX2fQXKRe4D7w2b,Ye,24,24,2,...,False,ye,1,Ye,None,None,None,None,None,None
2,0,11,"$, Kanye West, Ty Dolla $ign",BEG FORGIVENESS,89,spotify:track:4ihV1wv7QmjbkLHaT9lA4s,Ye,89,-1,1,...,False,ye,1,Ye,None,None,None,None,None,None
3,0,12,"$, Kanye West, Ty Dolla $ign",BURN,21,spotify:track:04CyMEHliadfQWMUJb1w99,Ye,21,-1,1,...,False,ye,1,Ye,None,None,None,None,None,None
4,1,17,"$, Kanye West, Ty Dolla $ign",BURN,36,spotify:track:04CyMEHliadfQWMUJb1w99,Ye,21,21,2,...,False,ye,1,Ye,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86777,42,42,"zzoilo, Aitana",Mon Amour - Remix,178,spotify:track:5TMLavqGRKNxgzMBrVpC9R,Universal Music Spain S.L.,34,199,43,...,False,univers music spain,1,Universal Music Spain S.L.,None,None,None,None,None,None
86778,43,43,"zzoilo, Aitana",Mon Amour - Remix,198,spotify:track:5TMLavqGRKNxgzMBrVpC9R,Universal Music Spain S.L.,34,178,44,...,False,univers music spain,1,Universal Music Spain S.L.,None,None,None,None,None,None
86779,44,44,"zzoilo, Aitana",Mon Amour - Remix,191,spotify:track:5TMLavqGRKNxgzMBrVpC9R,Universal Music Spain S.L.,34,198,45,...,False,univers music spain,1,Universal Music Spain S.L.,None,None,None,None,None,None
86780,45,45,"zzoilo, Aitana",Mon Amour - Remix,184,spotify:track:5TMLavqGRKNxgzMBrVpC9R,Universal Music Spain S.L.,34,191,46,...,False,univers music spain,1,Universal Music Spain S.L.,None,None,None,None,None,None


#### we remove punctuation, i.e. remove any character that is not a word or space character with nothing

#### common legal business suffixes, using a package called cleanco

#### removing all capital letters

#### replace non-ASCII characters

#### remove the most common words using regular expressions.

#### Cosine Similarity

#### Fuzzy String Matching

#### Post Processing

#### NameMatcher

In [41]:
a = ['CocaCola','CocaCola 3 Oz','Twix','Twix Caramel','Foldgers 3 Oz','Foldgers 10 Oz','Haagen Dazs Caramel','Black Forest Ham','Black Label Whiskey']

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
cv=CountVectorizer()
vect=cv.fit_transform(all_sources)
km=KMeans(n_clusters=500)

In [42]:
vectorised_names = km.fit_predict(vect)

In [43]:
len(vectorised_names)

86782

In [44]:
all_files_za["source vectorised"] = vectorised_names

In [45]:
all_files_za["source vectorised"].unique().tolist()

[466,
 161,
 37,
 2,
 142,
 482,
 14,
 313,
 87,
 97,
 311,
 277,
 88,
 393,
 140,
 8,
 112,
 326,
 473,
 25,
 195,
 174,
 388,
 371,
 184,
 176,
 28,
 115,
 34,
 254,
 332,
 382,
 91,
 99,
 143,
 179,
 50,
 197,
 49,
 18,
 301,
 81,
 205,
 12,
 295,
 29,
 4,
 126,
 367,
 325,
 16,
 216,
 234,
 236,
 165,
 181,
 15,
 144,
 163,
 394,
 187,
 10,
 7,
 53,
 430,
 464,
 219,
 439,
 229,
 6,
 30,
 152,
 320,
 330,
 490,
 45,
 194,
 168,
 227,
 209,
 217,
 200,
 404,
 3,
 21,
 257,
 396,
 40,
 117,
 182,
 287,
 403,
 32,
 220,
 43,
 281,
 488,
 104,
 26,
 477,
 1,
 74,
 323,
 304,
 359,
 498,
 224,
 492,
 119,
 286,
 381,
 495,
 255,
 44,
 288,
 33,
 347,
 77,
 352,
 127,
 79,
 154,
 410,
 366,
 70,
 121,
 327,
 57,
 59,
 56,
 134,
 218,
 73,
 94,
 63,
 462,
 392,
 340,
 106,
 232,
 169,
 206,
 22,
 468,
 365,
 372,
 487,
 300,
 293,
 68,
 312,
 341,
 186,
 149,
 375,
 401,
 294,
 71,
 417,
 370,
 196,
 385,
 406,
 253,
 47,
 65,
 427,
 455,
 397,
 305,
 17,
 151,
 379,
 363,
 242,
 442,
 26

In [46]:
all_files_za[["source", 'source vectorised']]

,source,source vectorised
0,Ye,466
1,Ye,466
2,Ye,466
3,Ye,466
4,Ye,466
...,...,...
86777,Universal Music Spain S.L.,301
86778,Universal Music Spain S.L.,301
86779,Universal Music Spain S.L.,301
86780,Universal Music Spain S.L.,301


In [47]:
all_files_za[all_files_za['source vectorised'] == 69][["source", 'source vectorised']]

,source,source vectorised
41490,Rock The World/IDJ/Kanye LP7,69
41491,Rock The World/IDJ/Kanye LP7,69
41503,Rock The World/IDJ/Kanye LP7,69
41504,Rock The World/IDJ/Kanye LP7,69
41505,Rock The World/IDJ/Kanye/LP6,69
...,...,...
41732,Rock The World/IDJ/Kanye LP7,69
41733,Rock The World/IDJ/Kanye LP7,69
41734,Rock The World/IDJ/Kanye LP7,69
41748,Rock The World/IDJ/Kanye LP7,69


In [48]:
all_files_za.to_csv("source_vect.csv", index=False)